# Process optimization by genetic algorithm

Smaller size version

In [22]:
include("../src/ProcGA.jl")

Main.ProcGA

## reading data

In [5]:
using CSV
using DelimitedFiles
using DataFrames
using Missings

In [6]:
ptbl = CSV.read("proctable.csv",header = 1);

In [7]:
# take smaller size 
ptbl = ptbl[1:6,:];

In [8]:
mtbl = convert(Array, ptbl[1:2])

┌ Warning: indexing with colon as row will create a copy in the future use df[col_inds] to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1


6×2 Array{Any,2}:
 "BB105 間管"       50
 "BB201-3 3補助管2"  50
 "BH301M 間管"      50
 "BH301M 枝管1"     50
 "BH301M 枝管2"     50
 "EP201 間管"       50

In [18]:
ptbl2 = [coalesce.(x,0) for x in convert(Array, ptbl[3:end])]

6×14 Array{Int64,2}:
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2

In [10]:
s = string.(names(ptbl)[3:end])

14-element Array{String,1}:
 "pipecut"           
 "anneal_bs_prebend" 
 "anneal_ns_prebend" 
 "icing"             
 "rotbend"           
 "pressbend"         
 "anneal_bs_precalib"
 "anneal_ns_precalib"
 "vcalib"            
 "hcalib"            
 "buff_precut"       
 "mc_cut"            
 "buff_postcut"      
 "qc_check"          

In [11]:
nametable = DataFrame(id = 1:length(s), nm = s)

,id,nm
,Int64,String
1,1,pipecut
2,2,anneal_bs_prebend
3,3,anneal_ns_prebend
4,4,icing
5,5,rotbend
6,6,pressbend
7,7,anneal_bs_precalib
8,8,anneal_ns_precalib
9,9,vcalib


Read grouping info.

In [12]:
gtbl = readdlm("grouping.csv",',', skipstart = 1)

2×4 Array{Any,2}:
 "anneal_bs"  "[2;7]"  300  3
 "anneal_ns"  "[3;8]"  300  3

In [13]:
gtbl2=[]
for k in 1:size(gtbl)[1]
    x = gtbl[k, 2:end]
    push!(gtbl2,Dict(:id=>eval(Meta.parse(x[1])),:cnt=>x[2],:timespan=>x[3]))
end

In [14]:
gtbl2

2-element Array{Any,1}:
 Dict{Symbol,Any}(:timespan=>3,:id=>[2, 7],:cnt=>300)
 Dict{Symbol,Any}(:timespan=>3,:id=>[3, 8],:cnt=>300)

## Initialize ProcGA tables.

In [23]:
ProcGA.settable(ptbl2, gtbl2, mtbl)

true

## Initialize population

In [27]:
g1 = ProcGA.jobtablebase()

6×72 Array{Int64,2}:
 1  7  7  7  9  9  12  12  13  13  14  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14  …  0  0  0  0  0  0  0  0  0  0  0  0

In [29]:
ProcGA.validatejob!(g1,1000)

6×72 Array{Int64,2}:
 1  7  7  7  9  9  12  12  13  13  14  …  0   0  0   0  0  0  0   0  0   0
 0  1  7  7  7  0   9   9  12  12  13     0   0  0   0  0  0  0   0  0   0
 0  0  0  0  0  1   0   0   0   0   7     0   0  0   0  0  0  0   0  0   0
 0  0  0  0  0  0   0   0   1   0   0     0   0  0   0  0  0  0  14  0  14
 0  0  0  0  1  0   7   0   0   7   7     0   0  0   0  0  0  0   0  0   0
 0  0  0  1  0  0   0   0   0   0   0  …  0  14  0  14  0  0  0   0  0   0

In [30]:
ProcGA.checkvalidity(g1)

true

In [31]:
ProcGA.penalty(g1)

228

In [42]:
gn1 = ProcGA.jobtableshuffle()

6×72 Array{Int64,2}:
 12  7   0  0   0  0  0   0  0  0  0  …  0   0   0  0   0  0  0  0   0   0
  1  0  12  0   0  0  0  13  0  0  0     0   0   0  0   0  0  0  0  13   0
  1  7   0  0  13  7  0   0  0  0  0     0  14   0  0   0  0  0  0   0   0
 13  0   0  0   0  0  0   0  0  0  0     0   0  12  0   0  0  0  1  13  14
  0  0   0  0   0  7  0   0  0  0  0     0   0   0  0  14  0  0  0  12  12
  0  0   0  0   0  0  0   0  1  0  0  …  0   0  13  0  13  0  0  0   0   0

In [43]:
ProcGA.shrinkjob!(gn1)

6×72 Array{Int64,2}:
 12  7   0   0  0   0  0   0   0  0  9  …  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  12   0  0  13  0  14   0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
  1  7   0  13  7   0  0   0   0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 13  0   0   0  0   0  0   0   0  0  7     0  0  0  0  0  0  0  0  0  0  0  0
  0  0   0   0  7   0  0   0  14  7  0     0  0  0  0  0  0  0  0  0  0  0  0
  0  0   0   0  0   0  1   0   0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0

In [44]:
ProcGA.validatejob!(gn1,10000)

6×72 Array{Int64,2}:
 1  7  0  0  0  0  0   0   0   0  7  …  0   0   0  0   0  0   0   0  0   0
 0  1  7  7  7  9  9  12  12  13  0     0   0   0  0   0  0   0   0  0   0
 0  0  0  0  0  0  0   0   0   0  0     0   0   0  0   0  0  13  14  0  14
 0  0  0  0  0  1  0   0   7   7  0     0  14   0  0   0  0   0   0  0   0
 0  0  0  0  0  0  1   0   0   7  0     0   0  14  0   0  0  14   0  0   0
 0  0  0  1  0  0  0   0   0   0  0  …  0   0   0  0  14  0   0   0  0   0

In [45]:
ProcGA.checkvalidity(gn1)

true

In [46]:
ProcGA.penalty(gn1)

237

Make population

In [74]:
ppl = ProcGA.initpopulation(100);

In [33]:
vlst = []

0-element Array{Any,1}

## Evolution!

Due to slightly large data, validation can not be done completely under max iteration = 100.

In [47]:
# Anyway move generation
vl = ProcGA.evolution!(ppl, 500);

i:10 => (31, 31.0, 110)
i:20 => (31, 31.0, 81)
i:30 => (31, 31.0, 91)
i:40 => (31, 31.0, 92)
i:50 => (31, 31.0, 88)
i:60 => (31, 31.0, 31)
i:70 => (31, 31.0, 240)
i:80 => (31, 31.0, 83)
i:90 => (31, 31.0, 38)
i:100 => (31, 31.0, 227)
i:110 => (31, 31.0, 220)
i:120 => (31, 31.0, 50)
i:130 => (31, 31.0, 31)
i:140 => (31, 31.0, 270)
i:150 => (31, 31.0, 119)
i:160 => (31, 31.0, 280)
i:170 => (31, 31.0, 255)
i:180 => (31, 31.0, 31)
i:190 => (31, 31.0, 219)
i:200 => (31, 31.0, 99)
i:210 => (31, 31.0, 110)
i:220 => (31, 31.0, 31)
i:230 => (31, 31.0, 212)
i:240 => (31, 31.0, 72)
i:250 => (31, 31.0, 46)
i:260 => (31, 31.0, 198)
i:270 => (31, 31.0, 102)
i:280 => (31, 31.0, 78)
i:290 => (31, 31.0, 31)
i:300 => (31, 31.0, 286)
i:310 => (31, 31.0, 125)
i:320 => (31, 31.0, 112)
i:330 => (31, 31.0, 46)
i:340 => (31, 31.0, 262)
i:350 => (31, 31.0, 110)
i:360 => (31, 31.0, 256)
i:370 => (31, 31.0, 31)
i:380 => (31, 31.0, 297)
i:390 => (31, 31.0, 199)
i:400 => (31, 31.0, 45)
i:410 => (31, 31.0, 222)
i:4

In [48]:
vlst = vcat(vlst,vl);

In [49]:
v = ProcGA.penalty.(ppl)
idm = findmin(v)[2]
bestgen = ppl[idm]

6×72 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  0  0  0  0  0   9   9   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7  …  0  0  0  0  0  0  0  0  0  0  0  0

In [50]:
ProcGA.checkvalidity(bestgen)

true

In [51]:
ProcGA.penalty(bestgen)

31

In [52]:
ProcGA.clipjob(bestgen)

6×31 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …   0   0   0   0   0   0   0   0   0
 1  7  7  7  0  0  0  0  0   9   9   0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  1  0  7  7   7   0   0      0  13  13  14  14   0   0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0     13   0   0   0   0  14  14   0   0
 0  0  0  1  0  0  0  7  7   7   0   9      0  14  14   0   0   0   0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7  …  12   0   0  13  13   0   0  14  14

In [54]:
ProcGA.validlength(bestgen)

31

Sort it by first appearance of process.

In [69]:
bestgensort = similar(bestgen)
for i in 1:size(bestgen)[1]
    idx = findfirst(x->(x>0),bestgen[i,:])
    bestgensort[idx,:] = bestgen[i,:]
end
bestgensort

6×72 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0

In [70]:
# save best process table to file.
# do not clip it so that it can be used again
writedlm("bestprc_20181112.csv",bestgensort, ',')

## read from saved data

In [40]:
rdlst = readdlm("bestprc_20181112.csv", ',', Int)

6×72 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  0  0  0  0  0   9   9   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7  …  0  0  0  0  0  0  0  0  0  0  0  0

In [41]:
ppl = ProcGA.initpopulationfrom(rdlst,100);

## Conclusion

For given 6 same data with proctalbe below, the best combination is "bestgen" here.

Its length is 31.
Taking time unit as 1 hour and assuming job time as 7 hours/day, it needs almost 4 days to finish these materials.

In [58]:
transpose(ProcGA.proctable[1,:])

1×14 LinearAlgebra.Transpose{Int64,Array{Int64,1}}:
 1  0  0  0  0  0  3  0  2  0  0  2  2  2

In [59]:
ProcGA.clipjob(bestgen)

6×31 Array{Int64,2}:
 0  0  1  0  7  7  7  9  9  12  12  13  …   0   0   0   0   0   0   0   0   0
 1  7  7  7  0  0  0  0  0   9   9   0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  1  0  7  7   7   0   0      0  13  13  14  14   0   0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0     13   0   0   0   0  14  14   0   0
 0  0  0  1  0  0  0  7  7   7   0   9      0  14  14   0   0   0   0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7  …  12   0   0  13  13   0   0  14  14